# ML-mетрики качества

*Хороши ли рекомендации, предлагаемые моделью?*  
Обычно считаются для каждого юзера, затем усредняются по юзерам

Представим, что в магазине всего 10 товаров

In [1]:
import numpy as np
import math

In [2]:
recommended_list = [143, 523, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [143, 523, 1134, 991]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [3]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [4]:
hit_rate(recommended_list, bought_list)

1

In [5]:
# def hit_rate_at_k(recommended_list, bought_list, k=5):
#     return hit_rate(recommended_list[:k], bought_list)

In [6]:
hit_rate_at_k(recommended_list, bought_list, k=2)

1

In [7]:
recommended_list

[143, 523, 1134, 991, 27, 1543, 3345, 533, 11, 43]

In [8]:
bought_list

[143, 523, 1134, 991]

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [9]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [10]:
import random

prices_recommended = dict()
keys = set(bought_list + recommended_list)
keys
for key in keys:
    prices_recommended[key] = random.randint(10, 1000)

In [11]:
prices_recommended

{32: 725,
 1543: 358,
 521: 390,
 11: 593,
 43: 386,
 1134: 772,
 143: 849,
 3345: 309,
 533: 191,
 27: 399,
 156: 87,
 991: 579}

In [12]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    bought_sum = [prices_recommended[product] for product in bought_list]
    recommended_sum = sum([prices_recommended[product] for product in recommended_list])
    
    precision = np.dot(flags, bought_sum) / recommended_sum    
    
    return precision

In [13]:
precision(recommended_list, bought_list)

0.2

In [14]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [15]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=3)

0.4970725995316159

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [16]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    bought_sum = [prices_recommended[product] for product in bought_list]
    
    recall = np.dot(flags, bought_sum) / sum(bought_sum)
    
    return recall

In [17]:
recall(recommended_list, bought_list)

0.5

In [18]:
recall_at_k(recommended_list,bought_list, k=10)

0.5

In [19]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.5615414864333464

---

# Метрики ранжирования

Metrics: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Если важен порядок рекомендаций.

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [20]:
recommended_list = [221,2,3234,1,234,234,234,666] #id товаров
bought_list = [1,2,3,4,5,6,7,8,9]

In [21]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant
  
    

In [22]:
bought_list = np.array(bought_list)

In [23]:
bought_list

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [24]:
recommended_list = np.array(recommended_list)[:5]

In [25]:
recommended_list

array([ 221,    2, 3234,    1,  234])

In [26]:
relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]

In [27]:
relevant_indexes

array([1, 3], dtype=int64)

In [28]:
amount_relevant = len(relevant_indexes)

In [29]:
amount_relevant

2

In [30]:
precision_at_k(recommended_list, bought_list, k=2)

0.5

In [31]:
precision_at_k(recommended_list, bought_list, k=4)

0.5

In [32]:
sum_ = np.sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])

In [33]:
sum_

1.0

In [34]:
sum_/amount_relevant

0.5

In [35]:
ap_k(recommended_list, bought_list, k=5)

0.5

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [36]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [37]:
recommended_list_3_users[0]

[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]

In [38]:
def map_k(recommended_list, bought_list, k=5):
    recommended_list = recommended_list[:k]
    sum_bought = 0
    count_users = len(recommended_list_3_users)
    
    for num in range(count_users):
        sum_bought += ap_k(recommended_list[num], bought_list[num], k=5)
    
    result = sum_bought / count_users
    
    return result

In [39]:
map_k(recommended_list_3_users, bought_list_3_users, 3)

0.3333333333333333

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


In [40]:
1, 0, 0, 1, 0 

(1, 0, 0, 1, 0)

$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [41]:
# по желанию
def ndcg_at_k(recommended_list, bought_list, k=5):
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    
    discount = []
    for num in range(1, len(recommended_list) + 1):
        if num <= 2:
            discount.append(num)
        else:
            discount.append(math.log2(num))
#     discount = [math.log2(i) for i in range(1, len(recommended_list) + 1)]
    
    DCG = np.dot(flags, discount) / len(recommended_list)
    NDCG = DCG / np.dot(np.ones(len(recommended_list)), discount)
    
    return NDCG

In [42]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 143]] # юзер 3

In [43]:
ndcg_at_k(recommended_list_3_users[0], bought_list_3_users[0], 5)

0.022454525274915656

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [44]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    rank = []
    recommended_list = recommended_list[:k]
    for num in range(len(recommended_list)):
        recommend = recommended_list[num]
        bought = bought_list[num]
        flags = np.isin(bought, recommend)
        for elem in flags:
            if elem == True:
                rank.append(list(flags).index(True) + 1)
                continue
    result = np.mean(rank)
    return result

In [45]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

2.5

# 3. Связь бизнес-метрик, ML-метрик и функции потерь(loss)

- **loss** - то, что оптимизирует модель (RMSE - Root Mean Squared Error)
- **ML-метрика** - то, как мы измеряем качество на test (money precision@5)
- **Бизнес-метрика** - то, что хочет оптимизировать бизнес (выручка)

В идеале loss = ML-метрика = бизнес-метрика, но это возмонжо в очень редких случаях

Надеемся, что:  
    *Оптимизация loss --> рост ML-метрик --> рост бизнес-метрик*

### Пример: рекомендательная система товаров на сайте  
      
На сайте есть баннер, на котором мы можем разместить 5 товаров. Надо порекомендовать эти 5 товаров персонально каждому юзеру

*Шаг 1: Определим бизнес-метрику*  
Бизнес хочет максимизировать выручку --> бизнес-метрика - **Выручка**  

*Шаг 2: Разложим ее на составляющие*  
Выручка =   
       Средний чек * кол-во покупок =   
       Средний чек * (число юзеров * конверсия из захода на сайт в заказ) =
       Число юзеров * (Средний чек * конверсия из захода на сайт в заказ)
       
Рекомендательная система влияет только на (Средний чек * конверсия из захода на сайт в заказ).   
Хорошим приближением этого является **money precision@5** - ML-метрика

*Шаг 3: loss*  
Это существенно сложнее. Персонализированные ML-модели не умеют напрямую оптимизировать конверсию. Стандартный loss - **RMSE**. Можно для начала попробовать его. 

P.S. Если вы хотите приблизить RMSE к money precision@5, то можно посчитать weighted RMSE, где вес каждого наблюдения = стоимость товара.   
P.S.S. Про другие виды loss будет рассказано в курсе

# Полезные ссылки

Статьи: https://docs.google.com/document/d/16L1u5zuQyT5rSAQNk2Q7Zk5KzqlPqWTpYa2cPK3QU0Y/edit?usp=sharing

# Домашнее задание 

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**2) Доделать все функции, где стоит комментарий "сделать дома"**

Примеры метрик для оценки рекомендаций:
1. MAE (Mean Absolute Error)	
2. MSE (Mean Squared Error)	
3. MSE (Root Mean Squared Error)

Примеры метрик для оценки ранжирования:

    1. Ранговый коэффициент корреляции Спирмена
    2. Ранговый коэффициент корреляции Кендэлла
    3. Expected reciprocal rank (ERR) 
    4. PFound 
    5. Fraction of Concordance Pairs